In [ ]:
import pandas as pd

# Cargar el dataset
file_path = 'C:\\Users\\angel\\OneDrive - Universidad Tecnológica de Panamá\\UTP\\Año IV\\Semestre I\\1. Análisis de Datos y Toma de Decisiones (ADTD)\\Proyecto Final\\Pokemon.csv'
pokemon_df = pd.read_csv(file_path)

# Mostrar las primeras filas del dataset para inspeccionar su estructura
pokemon_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo de gráficos
sns.set(style="whitegrid")

# Conteo de Pokémon por tipo
type_counts = pokemon_df['Type 1'].value_counts().reset_index()
type_counts.columns = ['Type', 'Count']

# Crear gráfico de barras para la distribución de tipos de Pokémon
plt.figure(figsize=(12, 6))
sns.barplot(x='Type', y='Count', data=type_counts, palette="viridis", hue='Type', legend=False)
plt.title('Distribución de Tipos de Pokémon')
plt.xlabel('Tipo de Pokémon')
plt.ylabel('Conteo')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Crear un gráfico de boxplot para cada estadística por generación
stats = ['HP', 'Atk', 'Def', 'Spa', 'Spd', 'BST']

plt.figure(figsize=(18, 12))
for i, stat in enumerate(stats, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x='Generation', y=stat, data=pokemon_df, palette="muted", hue='Generation', legend=False)
    plt.title(f'{stat} por Generación')
    plt.xlabel('Generación')
    plt.ylabel(stat)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular la matriz de correlación
corr_matrix = pokemon_df[stats].corr()

# Crear un heatmap para visualizar la matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title('Matriz de Correlación entre Estadísticas')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Preparar los datos
features = pokemon_df[stats]
labels = pokemon_df['Legendary']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Cargar el dataset
file_path = r'C:\Users\angel\OneDrive - Universidad Tecnológica de Panamá\UTP\Año IV\Semestre I\1. Análisis de Datos y Toma de Decisiones (ADTD)\Proyecto Final\Pokemon.csv'
pokemon_df = pd.read_csv(file_path)

# Verificar los nombres de las columnas
print(pokemon_df.columns)

# Inicializar la app Dash
app = dash.Dash(__name__)

# Layout de la app
app.layout = html.Div([
    html.H1('Dashboard de Análisis de Pokémon'),
    dcc.Tabs(id='tabs', children=[
        dcc.Tab(label='Distribución de Tipos', children=[
            dcc.Graph(id='bar-plot')
        ]),
        dcc.Tab(label='Estadísticas por Generación', children=[
            dcc.Dropdown(
                id='stat-dropdown',
                options=[{'label': stat, 'value': stat} for stat in ['HP', 'Atk', 'Def', 'Spa', 'Spd', 'Spe']],
                value='HP'
            ),
            dcc.Graph(id='box-plot')
        ]),
        dcc.Tab(label='Matriz de Correlación', children=[
            dcc.Graph(id='heatmap')
        ])
    ])
])

# Callbacks para actualizar los gráficos
@app.callback(
    Output('bar-plot', 'figure'),
    Input('tabs', 'value')
)
def update_bar_plot(tab):
    type_counts = pokemon_df['Type 1'].value_counts().reset_index()
    type_counts.columns = ['Type', 'Count']
    fig = px.bar(type_counts, x='Type', y='Count', title='Distribución de Tipos de Pokémon')
    return fig

@app.callback(
    Output('box-plot', 'figure'),
    Input('stat-dropdown', 'value')
)
def update_box_plot(stat):
    fig = px.box(pokemon_df, x='Generation', y=stat, title=f'{stat} por Generación')
    return fig

@app.callback(
    Output('heatmap', 'figure'),
    Input('tabs', 'value')
)
def update_heatmap(tab):
    stats = ['HP', 'Atk', 'Def', 'Spa', 'Spd', 'Spe']
    corr_matrix = pokemon_df[stats].corr()
    fig = px.imshow(corr_matrix, title='Matriz de Correlación entre Estadísticas')
    return fig

# Ejecutar la app
if __name__ == '__main__':
    app.run_server(debug=True)